In [2]:
import sys
sys.path.append('..')

In [3]:
from cnnfeat import get_feats
import keras
from keras.models import Model
from keras.optimizers import Adam, SGD
from keras.regularizers import l2
from keras.layers import *
import pandas as pd
import numpy as np
from position import Position
import matplotlib.pyplot as plt
%matplotlib inline
import imp
# imp.reload(cnnfeat)
from multiprocessing import Queue
from multiprocessing import Manager, Process, Pool
import os
from time import time
import pickle
import bcolz
from tqdm import tqdm 

Initializing Magics


Using TensorFlow backend.


In [4]:
import bcolz
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
from tqdm import tqdm
from time import time

In [5]:
# file_name1 = '../slonik_data/positions.pkl'
# file_name2 = '../slonik_data/sf_scores.pkl'
# data = pd.concat([pd.read_pickle(file_name1), pd.read_pickle(file_name2)])
file_name = '/ssd/slonik_data/sf_move_scores_4dep.pkl'
data = pd.read_pickle(file_name)

In [12]:
def fen_to_feats(i):
    return get_feats(Position.from_fen(data.iloc[i].fen))

In [13]:
len(data)

690435

In [ ]:
now = time()
n = len(data)
n_train = len(data) - 60000
n_valid = 60000
step=1000000
for i in range(0, n_train, step):
    for j in range(i, min(i+step, n_train)):
        features.append(fen_to_feats(j))
    features.flush()
for i in range(n_train, n, step):
    for j in range(i, min(i+step, n)):
        features_valid.append(fen_to_feats(j))
    features_valid.flush()    
print((time() - now) / 60, 'm')

In [14]:
move_scores = pd.read_pickle('/ssd/slonik_data/sf_move_scores_4dep.pkl')

# for num_workers in range(1,11):
features = bcolz.carray(np.empty((0,8,8,14)), rootdir='/ssd/slonik_data/features_move_scores_sf4_origfen.bc', chunklen=64, mode='w')
features_valid = bcolz.carray(np.empty((0,8,8,14)), rootdir='/ssd/slonik_data/features_move_scores_sf4_origfen_valid.bc', chunklen=64, mode='w')

n = len(move_scores)
n_valid = 60000
n_train = n - n_valid
step = 1024 * 8

#now = time()

def do_work(i):
    return get_feats(Position.from_fen(move_scores.iloc[i].orig_fen))

for s in tqdm(range(0, n_train, step)):
    num_workers = 7

    with Pool(num_workers) as p:
        res = p.map(do_work, range(s, min(s + step, n_train)))

    features.append(np.stack(res))
    features.flush()
    
for s in tqdm(range(n_train, n, step)):
    num_workers = 7

    with Pool(num_workers) as p:
        res = p.map(do_work, range(s, min(s + step, n)))

    features_valid.append(np.stack(res))
    features_valid.flush()

# print(num_workers, "workers", len(features_sf12), "results", (time() - now), 's')

100%|██████████| 8/8 [00:08<00:00,  1.06s/it]


In [15]:
features = bcolz.open(rootdir='/ssd/slonik_data/features_move_scores_sf4_origfen.bc', mode='r')
print(len(features))
features_valid = bcolz.open(rootdir='/ssd/slonik_data/features_move_scores_sf4_origfen_valid.bc', mode='r')
print(len(features_valid))
move_scores = pd.read_pickle('/ssd/slonik_data/sf_move_scores_4dep.pkl')
len(move_scores)
# features_positions_sf3 = bcolz.open(rootdir='/ssd/slonik_data/features_positions+sf3.bc', mode='r')

630435
60000


690435

In [16]:
np.allclose(get_feats(Position.from_fen(move_scores.iloc[-60000 + 20].orig_fen)), features_valid[20])

True

In [17]:
np.allclose(get_feats(Position.from_fen(move_scores.iloc[1000].orig_fen)), features[1000])

True